# Analyse Tag from Flickr Search Results via flickrapi

Be sure to create a file `api_creds` and add two global variables `api_key` and `api_key`. See Flickr API: https://www.flickr.com/services/api/

In [ ]:
!pip install -r requirements.txt

In [1]:

from PIL import Image
import urllib
from functions.gallery import gallery
import requests
import tabulate
import numpy as np
from wordcloud import WordCloud
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import folium


In [2]:
import api_creds

In [3]:
import flickrapi

flickr=flickrapi.FlickrAPI(api_creds.api_key, api_creds.api_secret, cache=True, format='parsed-json')

In [ ]:
cluster_result = flickr.tags.getClusters(tag="Red Kite")
tags = [tag["_content"] for tag in cluster_result["clusters"]["cluster"][0]["tag"]]

In [ ]:
wordcloud = WordCloud(background_color="white").generate(" ".join(tags))

plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


In [ ]:
tabulate.PRESERVE_WHITESPACE = True
tabulate.tabulate(zip([i+1 for i in range(len(tags))],tags), tablefmt='html', headers=["Rank", "Tags"])

# Tag Analysis

### Comparison eBird:

- Sightings Milvus Milvus: 171.749
- Images: 7.997/4.858
(as of 24th of June 2021)

In [4]:
labels = ["Red Kite", "Milvus Milvus", "Bird", "Raptor", "Birdofprey", "Rotmilan", "Milan Royal", "Raubvogel", "Roter Milan", "Tree", "Baum", "Sky", "Himmel"]

In [ ]:
total_count = flickr.photos.search(tags=",".join(labels))["photos"]["total"]

In [ ]:
total_count

In [ ]:
counts = [int(flickr.photos.search(tags=label)["photos"]["total"]) for label in labels]

In [ ]:
geo_counts = [int(flickr.photos.search(tags=label, has_geo=1)["photos"]["total"]) for label in labels]

In [ ]:
counts

In [ ]:
geo_counts

In [ ]:
x = np.arange(len(labels[::-1]))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.barh(x + width/2, counts[::-1], width, label='#Results All')
rects2 = ax.barh(x - width/2, geo_counts[::-1], width, label='#Results Geo-Tagged')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Search Results')
ax.set_title('#Results for Specified Tag')
ax.set_yticks(x)
ax.set_yticklabels(labels[::-1])
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.savefig("global_counts")
plt.show()

**Total Percentage Geo-tagged:**

In [ ]:
np.mean(np.array(geo_counts)/np.array(counts))

**Milvus Milvus Percentage Geo-tagged:**

In [ ]:
geo_counts[1]/counts[1]

## Results in specified area

Further analyze results in specified area

In [28]:
bbox = [[46.88041, 8.3924], [47.24894, 8.80186] ]
m = folium.Map(location=[47.07,8.38],
                   tiles='Stamen Terrain', zoom_start = 6)

# Overlay raster called img using add_child() function (opacity and bounding box set)
m.add_child(folium.vector_layers.Rectangle(bounds =bbox_canton_zug))


In [ ]:
scottland_counts = [int(flickr.photos.search(tags=label, has_geo=1, bbox=",".join([str(bbox[0][1]), str(bbox[0][0]), str(bbox[1][1]), str(bbox[1][0])]))["photos"]["total"]) for label in labels]

In [ ]:
flickr.photos.search(has_geo=1, bbox=",".join([str(bbox[0][1]), str(bbox[0][0]), str(bbox[1][1]), str(bbox[1][0])]))["photos"]["total"]

In [ ]:
scottland_counts

In [ ]:
x = np.arange(len(labels[::-1]))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.barh(x + width/2, scottland_counts[::-1], width, label='#Results All')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Search Results')
ax.set_title('#Results for Specified Tag')
ax.set_yticks(x)
ax.set_yticklabels(labels[::-1])
ax.legend()

ax.bar_label(rects1, padding=3)

fig.tight_layout()

plt.savefig("tags_zug_und_schwyz")
plt.show()